# Image Filtering

## Overview

### Learning Objectives

* Understand the image processing pipeline model used in ITK
* Understand what image streaming is, and how it is used in ITK
* Gain familiarity with some of the filtering algorithms available in ITK

While **data objects** (e.g., [Images](https://itk.org/ITKSoftwareGuide/html/Book1/ITKSoftwareGuide-Book1ch4.html#x40-440004.1) and [Meshes](https://itk.org/ITKSoftwareGuide/html/Book1/ITKSoftwareGuide-Book1ch4.html#x40-590004.3)) are used to represent data, **process objects** are classes that operate on data objects and may produce new data objects.

Process objects are classed as sources, filter objects, or mappers.

**Sources** (such as readers) produce data, **filter** objects take in data and process it to produce new data, and **mappers** accept data for output either to a file or some other system. 

Sometimes the term *filter* is used broadly to refer to all three types.

![Data pipeline](data/data-pipeline.png)

Typically **data objects** and **process objects** are connected together using the `SetInput()` and `GetOutput()` methods.

Generation of new outputs and pixel data does not occur until the `Update()` method is called on the **end** of the pipeline (on the process object or the data object).

![Pipeline updates](data/pipeline-updates.png)

The **data** associated with multi-dimensional images **is large and becoming larger**. 

Any practical image analysis system must address this fact in order to be useful in applications with multi-dimensional images. ITK addresses this problem via its **data streaming** facility.

![Streaming](data/streaming.gif)

Streaming is performed by splitting the image into non-overlapping **regions** at the **end of the pipeline**. The *RequestedRegion* then propagates up the pipeline.



There are three named [ImageRegion](https://itk.org/Insight/Doxygen/html/classitk_1_1ImageRegion.html)'s encountered in ITK:

- *BufferedRegion*: The region of pixels stored in memory

- *LargestPossibleRegion*: The largest possible region of an image

- *RequestedRegion*: The region requested on a single processing pass when streaming. The BufferedRegion and LargestPossibleRegion are as large or larger than the RequestedRegion

For more information, see,

- The [*Filtering* chapter](https://itk.org/ITKSoftwareGuide/html/Book2/ITKSoftwareGuide-Book2ch2.html#x17-320002) of Book 2 of the ITK Software Guide.
- The [*Data Processing Pipeline* section](https://itk.org/ITKSoftwareGuide/html/Book1/ITKSoftwareGuide-Book1ch3.html#x39-420003.5) of the *System Overview* chapter of Book 1 of the ITK Software Guide.
- The [*Data Representation* chapter](https://itk.org/ITKSoftwareGuide/html/Book1/ITKSoftwareGuide-Book1ch4.html#x45-490004) of Book 1 of the ITK Software Guide.

## Tutorial

In [1]:
import numpy as np
import itk
from itkwidgets import view

We can monitor when a filter gets called by registering a command that gets called when [ProgressEvents](https://itk.org/Doxygen/html/classitk_1_1ProgressEvent.html) occur.


In [2]:
itk.auto_progress(2)

Start an image processing pipeline with a source, an `ImageFileReader`.

In [28]:
fileName = 'data/PacMan.png'

InputPixelType = itk.F
OutputPixelType = itk.F
Dimension = 2

InputImageType = itk.Image[InputPixelType, Dimension]
OutputImageType = itk.Image[OutputPixelType, Dimension]

reader = itk.ImageFileReader[InputImageType].New()
reader.SetFileName(fileName)

Next, let's create a smoothing filter. To connect the pipeline, specify the `Output` of the reader as the `Input` to the smoother.

In [29]:
smoother = itk.RecursiveGaussianImageFilter[InputImageType,OutputImageType].New()
smoother.SetInput(reader.GetOutput())


At this point, no output images have been generated.

We have configured the simple pipeline:

reader -> smoother

In [30]:
print("reader's Output: %s" % reader.GetOutput())
print("smoother's Output: %s" % smoother.GetOutput())

reader's Output: Image (0x69ebee0)
  RTTI typeinfo:   itk::Image<float, 2u>
  Reference Count: 3
  Modified Time: 3700
  Debug: Off
  Object Name: 
  Observers: 
    none
  Source: (0x211dc10) 
  Source output name: Primary
  Release Data: Off
  Data Released: False
  Global Release Data: Off
  PipelineMTime: 0
  UpdateMTime: 0
  RealTimeStamp: 0 seconds 
  LargestPossibleRegion: 
    Dimension: 2
    Index: [0, 0]
    Size: [0, 0]
  BufferedRegion: 
    Dimension: 2
    Index: [0, 0]
    Size: [0, 0]
  RequestedRegion: 
    Dimension: 2
    Index: [0, 0]
    Size: [0, 0]
  Spacing: [1, 1]
  Origin: [0, 0]
  Direction: 
1 0
0 1

  IndexToPointMatrix: 
1 0
0 1

  PointToIndexMatrix: 
1 0
0 1

  Inverse Direction: 
1 0
0 1

  PixelContainer: 
    ImportImageContainer (0x415de90)
      RTTI typeinfo:   itk::ImportImageContainer<unsigned long, float>
      Reference Count: 1
      Modified Time: 3697
      Debug: Off
      Object Name: 
      Observers: 
        none
      Pointer: 0
     

To generate the filter outputs, we must call `Update()` on the filter at the end of the pipeline. In this case, it is the smoother.

In [31]:
smoother.Update()

print("reader's Output: %s" % reader.GetOutput())
print("smoother's Output: %s" % smoother.GetOutput())

reader's Output: Image (0x69ebee0)
  RTTI typeinfo:   itk::Image<float, 2u>
  Reference Count: 3
  Modified Time: 4172
  Debug: Off
  Object Name: 
  Observers: 
    none
  Source: (0x211dc10) 
  Source output name: Primary
  Release Data: Off
  Data Released: False
  Global Release Data: Off
  PipelineMTime: 3711
  UpdateMTime: 4173
  RealTimeStamp: 0 seconds 
  LargestPossibleRegion: 
    Dimension: 2
    Index: [0, 0]
    Size: [128, 128]
  BufferedRegion: 
    Dimension: 2
    Index: [0, 0]
    Size: [128, 128]
  RequestedRegion: 
    Dimension: 2
    Index: [0, 0]
    Size: [128, 128]
  Spacing: [1, 1]
  Origin: [0, 0]
  Direction: 
1 0
0 1

  IndexToPointMatrix: 
1 0
0 1

  PointToIndexMatrix: 
1 0
0 1

  Inverse Direction: 
1 0
0 1

  PixelContainer: 
    ImportImageContainer (0x415de90)
      RTTI typeinfo:   itk::ImportImageContainer<unsigned long, float>
      Reference Count: 1
      Modified Time: 4170
      Debug: Off
      Object Name: 
      Observers: 
        none
    

Running itkImageFileReaderIF2... done
Running itkRecursiveGaussianImageFilterIF2IF2... done
done


Let's view the images.

In [33]:
image = reader.GetOutput()
view(image)

Viewer(gradient_opacity=0.22, rendered_image=<itkImagePython.itkImageF2; proxy of <Swig Object of type 'itkIma…

In [34]:
smoothed = smoother.GetOutput()
view(smoothed)

Viewer(gradient_opacity=0.22, rendered_image=<itkImagePython.itkImageF2; proxy of <Swig Object of type 'itkIma…

If we call `Update()` on the pipeline, the output pixel data is not needlessly generated because the pipeline is up-to-date.

In [35]:
smoother.Update()

However, if we change the amount of smoothing, new pixel data does need to be generated from the output of the smoother.

In [36]:
smoother.SetSigma(10.0)
smoother.Update()

Running itkRecursiveGaussianImageFilterIF2IF2... done
done


In [37]:
view(smoothed)

Viewer(gradient_opacity=0.22, rendered_image=<itkImagePython.itkImageF2; proxy of <Swig Object of type 'itkIma…

Note, however, note that the reader does generate its output because it is up-to-date and upstream from the smoother.

If we artificially modify the reader, both the reader and the smoother need to regenerate their outputs.

In [38]:
reader.Modified()

smoother.Update()

Running itkImageFileReaderIF2... done
Running itkRecursiveGaussianImageFilterIF2IF2... done
done


We can stream the pipeline by placing a [StreamingImageFilter](https://itk.org/Doxygen/html/classitk_1_1StreamingImageFilter.html) at the **end** of the pipeline. The smoother generates outputs multiple times, once for each image region streaming division. Since the reader is not capable of streaming, it only generates its output once.

In [39]:
streamer = itk.StreamingImageFilter.New(Input=smoother.GetOutput())
streamer.SetNumberOfStreamDivisions(3)
reader.Modified()
streamer.Update()

Running itkStreamingImageFilterIF2IF2... Running itkImageFileReaderIF2... done
Running itkRecursiveGaussianImageFilterIF2IF2... done
done
Running itkStreamingImageFilterIF2IF2... Running itkRecursiveGaussianImageFilterIF2IF2... done
done
Running itkRecursiveGaussianImageFilterIF2IF2... done
done
done


A file writer can also stream data if the output format supports it.

In [43]:
writer = itk.ImageFileWriter[OutputImageType].New()
writer.SetInput(smoother.GetOutput())
writer.SetFileName('my_output.mha')
num_write_subsessions=30
writer.SetNumberOfStreamDivisions(num_write_subsessions)
writer.Update()

Running itkRecursiveGaussianImageFilterIF2IF2... done
done
Running itkImageFileWriterIF2... Running itkRecursiveGaussianImageFilterIF2IF2... done
done
Running itkRecursiveGaussianImageFilterIF2IF2... done
done
Running itkRecursiveGaussianImageFilterIF2IF2... done
done
Running itkRecursiveGaussianImageFilterIF2IF2... done
done
Running itkRecursiveGaussianImageFilterIF2IF2... done
done
Running itkRecursiveGaussianImageFilterIF2IF2... done
done
Running itkRecursiveGaussianImageFilterIF2IF2... done
done
Running itkRecursiveGaussianImageFilterIF2IF2... done
done
Running itkRecursiveGaussianImageFilterIF2IF2... done
done
Running itkRecursiveGaussianImageFilterIF2IF2... done
done
Running itkRecursiveGaussianImageFilterIF2IF2... done
done
Running itkRecursiveGaussianImageFilterIF2IF2... done
done
Running itkRecursiveGaussianImageFilterIF2IF2... done
done
Running itkRecursiveGaussianImageFilterIF2IF2... done
done
Running itkRecursiveGaussianImageFilterIF2IF2... done
done
Running itkRecursiveGau

## Exercises

### Exercise 1: The effect of Sigma

Change the value of Sigma on the smoothing filter.

- How is the output effected?
- What are the units of Sigma?

In [ ]:
# fileName = 'data/PacMan.png'
# reader = itk.ImageFileReader[InputImageType].New()
# reader.SetFileName(fileName)
# smoother = itk.RecursiveGaussianImageFilter[InputImageType,OutputImageType].New()
# smoother.SetInput(reader.GetOutput())
# smoother.SetSigma(XX)
# smoother.Update()
# view(smoother.GetOutput())

In [17]:
# %load solutions/2_Image_Filtering_Exercise1.py

### Exercise 2: Does setting a filter parameter to its current value cause regeneration of its output?

Call `smoother.SetSigma(smoother.GetSigma())` then `smoother.Update()`. 

- Is the output regenerated? 
- Is this expected / desirable?

In [20]:
# %load solutions/2_Image_Filtering_Exercise2.py

### Exercise 3: Find other image filtering algorithms

The classes in ITK are organized into **Modules**, and collections of Modules are organized into **Groups**. Examine the [Image Smoothing](https://itk.org/Insight/Doxygen/html/group__ITKSmoothing.html) Module and the [Filtering](https://itk.org/Insight/Doxygen/html/group__Group-Filtering.html) Group. Can any other smoothing or denoising classes be found? 

In [23]:
# %load solutions/2_Image_Filtering_Exercise3.py

### Exercise 4: Test different output format with streaming

Not all file format support streaming. Test several of them and see which ones support it and which ones do not.
Hint 1: To select a file format, select the corresponding file extension.
Hint 2: Examples of file format: `jpg`, 


In [26]:
# %load solutions/2_Image_Filtering_Exercise4.py

### Enjoy ITK!